In [1]:
import os

In [2]:
%pwd

'c:\\Users\\harik\\OneDrive\\Desktop\\HARIKRISHNAN_DETAILS\\Real_Estate_Predictor_Web_App\\Real_Estate_Price_Predictor_Web_App\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\harik\\OneDrive\\Desktop\\HARIKRISHNAN_DETAILS\\Real_Estate_Predictor_Web_App\\Real_Estate_Price_Predictor_Web_App'

In [15]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    bucket_name: str
    local_data_file: Path
    s3_file_path: str
    amazon_s3_access_keys:str
    params_amazon_service_name:str

In [7]:
from real_estate_price_predictor.constants import *
from real_estate_price_predictor.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            bucket_name=config.bucket_name,
            local_data_file=config.local_data_file,
            s3_file_path=config.s3_file_path, 
            amazon_s3_access_keys = config.amazon_s3_access_keys,
            params_amazon_service_name= self.params.amazon_service_name
        )

        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import boto3
from real_estate_price_predictor import logger
from real_estate_price_predictor.utils.common import get_size
import pandas as pd

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            df = pd.read_csv(self.config.amazon_s3_access_keys)
            s3 = boto3.client(self.config.params_amazon_service_name,aws_access_key_id=df[df.columns[0]][0], aws_secret_access_key=df[df.columns[1]][0])
            s3.download_file(self.config.bucket_name, self.config.s3_file_path, self.config.local_data_file)
            logger.info(f"{self.config.local_data_file} download! with following info")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}") 

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2024-09-30 17:03:46,622: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-30 17:03:46,624: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-30 17:03:46,627: INFO: common: created directory at: artifacts]
[2024-09-30 17:03:46,629: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-30 17:03:51,189: INFO: 157159851: artifacts/data_ingestion/data.csv download! with following info]
